In [30]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras import  Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dropout, Dense, BatchNormalization, LSTM
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
import bnci_utils as utils

In [31]:
num_participants = 18
dataset_path = os.path.join('..', 'dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1))
         for i in range(num_participants)] # P01 - P18 files


# Set seed to produce consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [32]:
# Simple function to transform the dataset to be usable for the neural network
def transform_dataset(features, labels):
    labels = labels.reshape((-1, 1)) # reshape so one hot encoding can be used
    labels = OneHotEncoder().fit_transform(labels).toarray() # apply one hot encoding
    features = features.reshape((features.shape[0], 14, -1))

    return features, labels

In [33]:
# Definition of the lstm model
def lstm_model():
    model = Sequential([
        LSTM(124, input_shape=(14, 360), activation=tf.nn.relu, return_sequences=True),
        Dropout(0.4),
        BatchNormalization(),
        LSTM(124, activation=tf.nn.relu),
        Dropout(0.3),
        BatchNormalization(),
        Dense(64, activation=tf.nn.relu),
        Dropout(0.2),
        Dense(2, activation=tf.nn.softmax, name='output_layer')
    ])

    return model

In [34]:
# Function to run the network with training data and testing data
def run_network(model, x_train, y_train, x_test, y_test, iteration, epochs=30):

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy'],

    )
    # Train the model
    model.fit(x_train, y_train, epochs=epochs, callbacks=[
        EarlyStopping(patience=8, verbose=1, restore_best_weights=True, monitor='loss')
    ])

    # Get the results
    accuracy, precision, recall, f1, confusion_matrix = utils.get_metrics_keras(model, x_test, y_test,
                                                                                f'{iteration}. LSTM')
    # Return the results as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': confusion_matrix
    }

In [35]:
def run_individual(file, particip_num, model, test_size=0.25, epochs=30):
    print('Running ANN for file:', file)
    dataset = np.load(file) # load numpy file containing the preprocessed data for specific participant
    features, labels = dataset['features'], dataset['labels'] # get features and labels from the numpy file

    # transform numpy arrays
    features, labels = transform_dataset(features, labels)
    # split to training and testing data
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed,
                                                        shuffle=True)
    print('X (train) shape:', x_train.shape, 'Y (train) shape:', y_train.shape)
    print('X (test) shape:', x_test.shape, 'Y (test) shape:', y_test.shape)

    return run_network(model, x_train, y_train, x_test, y_test, particip_num, epochs)

In [36]:
i = 1
results = []
for file in files:
    results.append(run_individual(file=file, particip_num=i, model=lstm_model()))
    i += 1
    K.clear_session()

Running ANN for file: ..\dataset_result\P01.npz
X (train) shape: (90, 14, 360) Y (train) shape: (90, 2)
X (test) shape: (30, 14, 360) Y (test) shape: (30, 2)
Epoch 1/30
3/3 [==============================] - 5s 115ms/step - loss: 0.9746 - accuracy: 0.5062
Epoch 2/30
3/3 [==============================] - 0s 110ms/step - loss: 0.6656 - accuracy: 0.6314
Epoch 3/30
3/3 [==============================] - 0s 108ms/step - loss: 0.6794 - accuracy: 0.6308

In [37]:
# Create pandas dataframe with stats from each iterations
df = pd.DataFrame({
    'participant': [x for x in range(1, num_participants + 1)],
    'accuracy': [x['accuracy'] for x in results],
    'precision': [x['precision'] for x in results],
    'recall': [x['recall'] for x in results],
    'f1': [x['f1'] for x in results],
})

df

RESULTS:
0.6333333253860474
0.4523809552192688
0.5952380895614624
0.3571428656578064
0.4523809552192688
0.4047619104385376
0.5
0.5476190447807312
0.523809552192688
0.5
0.5
0.4047619104385376
0.4285714328289032
0.380952388048172
0.5
0.5476190447807312
0.380952388048172
0.4523809552192688
Average: 0.47566137876775527
Max: 0.6333333253860474


In [ ]:
utils.print_confusion_matrices(results)